In [256]:
import csv
import math
import re
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tweepy
from bs4 import BeautifulSoup
from keras.layers import Dense, Flatten, LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing import sequence
from nltk.corpus import stopwords   # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from textblob import TextBlob

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/guptasoumya/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [108]:
import pandas as pd
import datetime as dt
import numpy as np

## Import 4 months twitter data

In [175]:
with open('tweetwise_data/tweets_raw.txt','r') as f:
    text = f.read()

In [176]:
text1 = text.split("\n")

In [177]:
list1  = []
for i in text1:
    m = i.split("||")
    #print(m)
    if len(m) == 4:
        list1.append(m)
    

In [178]:
len(list1)

1047175

In [180]:
df = pd.DataFrame(list1)

In [181]:
df.drop([0,3], axis=1, inplace=True)

In [182]:
df.columns = ['Text','Date']

In [183]:
df.dtypes

Text    object
Date    object
dtype: object

In [184]:
df['Date'] = df['Date'].apply(lambda x: x.split('T'))
df = df[df['Date'].map(len) == 2]


In [185]:
df['Date']=df['Date'].apply(lambda x: x[0])

In [186]:
df = df.dropna()

In [233]:
df = df[df['Date'].isin(date_list)]

In [227]:
date_list = ['2016-10-27', '2016-10-28', '2016-10-29', '2016-10-30',
       '2016-10-31', '2016-11-01', '2016-11-02', '2016-11-03',
       '2016-11-04', '2016-11-05', '2016-11-06',
       '2016-11-07',  '2016-11-08',
       '2016-11-09', 
       '2016-11-10',
       '2016-11-11', '2016-11-12',  '2016-11-13',
       '2016-11-14', '2016-11-15', '2016-11-16', '2016-11-17',
       '2016-11-18', '2016-11-19',
       '2016-11-20',  '2016-11-21',  '2016-11-22','2016-11-23','2016-11-24','2016-11-25','2016-11-26', '2016-11-27','2016-11-28', '2016-11-29','2016-11-30',
       '2016-12-01',
       '2016-12-02',  '2016-12-03', '2016-12-04', 
       '2016-12-05', '2016-12-06', 
       '2016-12-07', '2016-12-08','2016-12-09','2016-12-10', '2016-12-11','2016-12-12', '2016-12-13', '2016-12-14',
       '2016-12-15', '2016-12-16', '2016-12-17', '2016-12-18','2016-12-19', '2016-12-20',
       '2016-12-21','2016-12-22', '2016-12-23','2016-12-24', '2016-12-25', '2016-12-26',
       '2016-12-27', '2016-12-28', '2016-12-29',
       '2016-12-30', '2016-12-31',
       '2017-01-01',  '2017-01-02','2017-01-03', '2017-01-04',  '2017-01-05', '2017-01-06',
       '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10',
       '2017-01-11','2017-01-12', '2017-01-13', '2017-01-14','2017-01-15',
       '2017-01-16',  '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20', '2017-01-21',  '2017-01-22', '2017-01-23', '2017-01-24','2017-01-25', '2017-01-26',
       '2017-01-27',  '2017-01-28', '2017-01-29', '2017-01-30',
       '2017-01-31',  '2017-02-01', '2017-02-02', '2017-02-03', '2017-02-04', '2017-02-05',  '2017-02-06', '2017-02-07', 
       '2017-02-08', '2017-02-09',  '2017-02-10', '2017-02-11', '2017-02-12', '2017-02-13', '2017-02-14', '2017-02-15', '2017-02-16', '2017-02-17',
       '2017-02-18', '2017-02-19', 
       '2017-02-20', '2017-02-21', '2017-02-22']

In [234]:
df['Date'] = df['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d') )
df['Date'] = df['Date'].dt.date

In [240]:
df.head() # final data with relevant date

,Text,Date
0,The price of bitcoin on @coinbase is now 673.6...,2016-10-27
1,RT @RedditBTC: Iceland: Few days before electi...,2016-10-27
2,Easy and easy and easy mining #bitcoin with cl...,2016-10-27
3,"Bitcoin based social media community, Yours, c...",2016-10-27
4,"RT @BitcoinMagazine: With TumbleBit, #Bitcoin ...",2016-10-27


## Import Bitcoin price data

In [241]:
btc = pd.read_csv('coindesk-bpi-USD-close_data-2010-07-17_2018-04-06.csv')

In [247]:
btc= btc[btc['Date']!='This data was produced from the CoinDesk price page.']
btc= btc[btc['Date']!='http://www.coindesk.com/price/']

In [248]:
btc['Date'] = btc['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') )
btc['Date'] = btc['Date'].dt.date

In [262]:
btc['Returns'] = btc['Close Pricee Price'].diff()

In [250]:
data = df.merge(btc, how='left', on='Date')

In [251]:
df.shape

(1045025, 2)

In [252]:
btc.shape

(2821, 2)

In [253]:
data.shape

(1045025, 3)

In [254]:
data.head() # Merged dataset

,Text,Date,Close Price
0,The price of bitcoin on @coinbase is now 673.6...,2016-10-27,686.52
1,RT @RedditBTC: Iceland: Few days before electi...,2016-10-27,686.52
2,Easy and easy and easy mining #bitcoin with cl...,2016-10-27,686.52
3,"Bitcoin based social media community, Yours, c...",2016-10-27,686.52
4,"RT @BitcoinMagazine: With TumbleBit, #Bitcoin ...",2016-10-27,686.52


## Clean twitter data

In [258]:
def beautify_tweet(raw_tweet):
    tweet_text = BeautifulSoup(raw_tweet, "lxml").get_text()
    letters_only = re.sub("[^a-zA-Z]"," ", tweet_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stopwords.words("english")]
    return( " ".join( meaningful_words ))

In [261]:
data['Text'] = data['Text'].apply( lambda x: beautify_tweet(x))

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/KZu2omLCYC" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/7BlqSSTWZ2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/WBjmkQyvJI" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/daAYkWubif" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/rqk2kx4Szk" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/AqOGWp9Bza" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/J5esJjH9no" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/laLeHApfrE" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/uoTByc…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document 

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/IRYqdfpxF3" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/lp6fa01J8J" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/OW9yCC7EBt" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/8EwsLp3nK…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/WVaU2gVdco" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/wllZUn90AX" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

...

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/JC8FOznlif" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/CooEc0FuDB" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/Vj1hAZYEt2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/kOTjVV57gm" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/TR3yHJvZnG" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/n5ik56TwKo" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docume

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/1wm9mIwFQX" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/fQqBWKV…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/N1COPRGTm4" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/RwPYv…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/DPgbIAAGld" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/HXz9EQVnn…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document t

/Users/guptasoumya/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "https://t.co/fYrL9hgAMt" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [275]:
"""for idx, row in data.iterrows():
    analysis = TextBlob(row['Text'])
    row ['Sentiment Score'] = analysis.sentiment.polarity
    if analysis.sentiment.polarity > 0:
        row['Sentiment'] =  1 # positive
    elif analysis.sentiment.polarity == 0:
        row['Sentiment'] =  0 # neutral
    else:
        row['Sentiment'] = -1 # negative"""

## Import Google trends data

In [277]:
gtrends = pd.read_csv('gtrends_bitcoin_buysell.csv')

In [289]:
gtrends.drop('Unnamed: 0',axis=1,inplace=True)

In [296]:
gtrends.head()

,Date,Gtrends Buy,Gtrends Sell
0,2016-10-27,100,20
1,2016-10-28,198,33
2,2016-10-29,152,39
3,2016-10-30,188,23
4,2016-10-31,193,16


In [379]:
gtrends['Date'] = gtrends['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d') )
gtrends['Date'] = gtrends['Date'].dt.date

## Sentiment analysis

In [282]:
def sentiment_score(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

In [283]:
data['Sentiment Score'] = data['Text'].apply(lambda x: sentiment_score(x))

In [299]:
df = data.merge(gtrends, how='left', on='Date')

In [297]:
data['Date'].iloc[0]

datetime.date(2016, 10, 27)

In [298]:
gtrends['Date'].iloc[0]

datetime.date(2016, 10, 27)

In [330]:
df.head()

,Date,Sentiment Score,Gtrends Buy,Gtrends Sell,Returns
0,2016-10-27,0.000000,100,20,12.16
1,2016-10-27,0.000000,100,20,12.16
2,2016-10-27,0.433333,100,20,12.16
3,2016-10-27,0.033333,100,20,12.16
4,2016-10-27,0.500000,100,20,12.16


In [306]:
df.drop('Close Price', axis=1, inplace=True)

In [317]:
df.drop('Text', axis=1, inplace=True)

In [304]:
df = df.merge(btc, how='left', on='Date')

In [351]:
sentiment = pd.DataFrame(df.groupby('Date').mean()['Sentiment Score']).reset_index()

In [352]:
master = sentiment.merge(gtrends, on='Date', how='left')

In [353]:
master = master.merge(btc, on='Date', how='left')

In [375]:
master.head()

,Unnamed: 0,Date,Sentiment Score,Gtrends Buy,Gtrends Sell,Close Price,Returns
0,0,2016-10-27,746.953975,100,20,686.52,12.16
1,1,2016-10-28,879.989770,198,33,688.10,1.58
2,2,2016-10-29,743.397193,152,39,712.87,24.77
3,3,2016-10-30,762.588867,188,23,696.35,-16.52
4,4,2016-10-31,840.353660,193,16,697.37,1.02


In [329]:
master.to_csv('master.csv')

In [374]:
master = pd.read_csv('master.csv')

In [355]:
#master.drop(['Date','Close Price'],axis=1, inplace=True)

In [357]:
master.describe()

,Sentiment Score,Gtrends Buy,Gtrends Sell,Returns
count,119.000000,119.000000,119.000000,119.000000
mean,0.086694,183.453782,27.789916,3.763277
std,0.015192,20.398051,10.306222,28.431757
min,0.040535,100.000000,5.000000,-128.810000
25%,0.075469,176.000000,20.500000,-4.495000
50%,0.088261,192.000000,27.000000,5.090000
75%,0.095219,198.500000,34.000000,13.480000
max,0.125299,200.000000,58.000000,99.050000


## Import S&P data 

In [366]:
snp = pd.read_csv('GSPC.csv')

In [369]:
snp.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-10-27,2144.060059,2147.129883,2132.520020,2133.040039,2133.040039,4204830000
1,2016-10-28,2132.229980,2140.719971,2119.360107,2126.409912,2126.409912,4019510000
2,2016-10-31,2129.780029,2133.250000,2125.530029,2126.149902,2126.149902,3922400000
3,2016-11-01,2128.679932,2131.449951,2097.850098,2111.719971,2111.719971,4532160000
4,2016-11-02,2109.429932,2111.760010,2094.000000,2097.939941,2097.939941,4248580000


In [368]:
snp['Date'] = snp['Date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d') )
snp['Date'] = snp['Date'].dt.date

In [370]:
snp = snp[['Date','Close']]

In [382]:
master.head()

,Unnamed: 0,Date,Sentiment Score,Gtrends Buy,Gtrends Sell,Close Price,Returns,Close
0,0,2016-10-27,746.953975,100,20,686.52,12.16,2133.040039
1,1,2016-10-28,879.989770,198,33,688.10,1.58,2126.409912
2,2,2016-10-29,743.397193,152,39,712.87,24.77,NaN
3,3,2016-10-30,762.588867,188,23,696.35,-16.52,NaN
4,4,2016-10-31,840.353660,193,16,697.37,1.02,2126.149902


In [381]:
master = master.merge(snp, how='left',on='Date')

In [378]:
master['Date'].iloc[0]

'2016-10-27'

## Normalize data

In [359]:
from sklearn import preprocessing

x = master.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [361]:
df.columns = master.columns

In [362]:
df.head()

,Sentiment Score,Gtrends Buy,Gtrends Sell,Returns
0,0.441453,0.00,0.283019,0.618669
1,0.646445,0.98,0.528302,0.572237
2,0.499848,0.52,0.641509,0.674010
3,0.604420,0.88,0.339623,0.492803
4,0.593931,0.93,0.207547,0.569780


In [363]:
df.describe()

,Sentiment Score,Gtrends Buy,Gtrends Sell,Returns
count,119.000000,119.000000,119.000000,119.000000
mean,0.544560,0.834538,0.429998,0.581819
std,0.179225,0.203981,0.194457,0.124777
min,0.000000,0.000000,0.000000,0.000000
25%,0.412135,0.760000,0.292453,0.545576
50%,0.563053,0.920000,0.415094,0.587642
75%,0.645132,0.985000,0.547170,0.624462
max,1.000000,1.000000,1.000000,1.000000
